In [ ]:
# パッケージのインストール
%pip install -q langchain
%pip install -q langchain-openai
%pip install -q langchain-chroma
%pip install -q langchain-community
%pip install -q python-dotenv
#%pip install -q python-magic-bin
#%pip install -q unstructured
%pip install -q pdfminer.six
%pip install -q chromadb
%pip install -q ragas datasets
%pip install -q matplotlib
%pip install -q ipywidgets


In [19]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [20]:
chunk_size = 200
collection_name = f"mycollection_{chunk_size}"
embedding_model = "text-embedding-3-large"

top_k = 10
batch_size = 500
search_type="similarity"
#search_type="similarity_score_threshold"
score_threshold = 0.2
query="レイジングハートとは何ですか？"

### LLMと埋め込みモデルの準備

In [21]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLMの準備
llm = ChatOpenAI(
    model="gpt-4o", # モデル
    temperature=0, # ランダムさ
)

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### ドキュメントの読み込みとチャンキング

In [22]:
from langchain_community.document_loaders import PDFMinerLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ドキュメントの読み込み
loader = DirectoryLoader(
    "./data/",
    glob="**/*.pdf",  # PDFファイルを対象に
    show_progress=True,
    loader_cls=PDFMinerLoader  # PDFMinerLoaderを使用
)

In [23]:
# ドキュメントの分割
documents = []
for doc in loader.load():
    # すべての改行を削除（単語の途中でチャンキングされることを防ぐため）
    doc.page_content = doc.page_content.replace("\n", "")
    chunks = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 10,
        separators=["。", "、", " "]
    ).split_documents([doc])
    documents.extend(chunks)

print(f"合計チャンク数: {len(documents)}")

100%|██████████| 19/19 [00:48<00:00,  2.55s/it]

合計チャンク数: 3375


In [24]:
for i, doc in enumerate(documents[:5]):
    print(f"\n=== Document {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")


=== Document 1 ===
Content: ハリー‧ポッター (架空の⼈物)出典: フリー百科事典『ウィキペディア（Wikipedia）』ハリー‧ポッターシリーズ > ハリー‧ポッターシリーズの登場⼈物⼀覧 > ハリー‧ポッター (架空の⼈物)ハリー‧ジェームズ‧ポッター（英: Harry JamesPotter）は、J‧K‧ローリングの⼩説『ハリー‧ポッター』シリーズおよび、その派⽣作品に登場する架空の⼈物であり、同シリーズの主⼈公
文字数: 197
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 2 ===
Content: 。ホグワーツ魔法魔術学校グリフィンドール寮の男⼦⽣徒となる。孤児として⺟親の親類の伯⺟夫婦の家で不遇な暮らしをして育った。11歳を迎える年のある⽇突然、ホグワーツから⼊学許可証が届いたのをきっかけに、亡くなった両親が魔法使いであったこと、そして出⽣時に下された予⾔により、闇の魔法使いヴォルデモートを倒す宿命を⾃分が負っていると告げられる
文字数: 169
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 3 ===
Content: 。マグル界では⼀介の少年に過ぎない⽣活を送っていたが、魔法界では本⼈が⼾惑うほど重要な⼈物として、あまねく⼈々から知られている。⼀⼈前の魔法使いになるべく、同級⽣のロン‧ウィーズリーやハーマイオニー‧グレンジャーらとともに、ホグワーツにて学⽣⽣活を送りつつ、宿敵のヴォルデモートなどの闇の魔法使いたちによる数々の陰謀に⽴ち向かう冒険の⽇々を通して、たくましく成⻑していく姿が物語で描かれている
文字数: 196
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 4 ===
Content: 。⼈物名前‧外⾒魔 法 界 で は 「 ⽣ き 残 っ た 男 の ⼦   (The  boy  wholived) 」と呼ばれる。髪の⽑は黒い癖⽑で、瞳は明るい緑⾊。⼩顔で細⾯で、近視のため丸眼鏡を着⽤。同年代に⽐べ⼩柄で痩せているが、第6巻『謎のプリンス』では前巻と⽐べて⾝⻑がかなり伸びたとされている
文字数: 1

### VectorStoreの準備

In [25]:
import chromadb
from langchain_chroma import Chroma


# Chromaクライアントを初期化
try:
    persistent_client = chromadb.PersistentClient(path="./chroma_db")
    # 既存のChromaコレクションを削除
    try:
        persistent_client.delete_collection(collection_name)
        print(f"既存のコレクション '{collection_name}' を削除しました")
    except ValueError as e:
        print(f"既存のコレクション '{collection_name}' は存在しませんでした")
    except Exception as e:
        print(f"コレクションの削除中にエラーが発生しました: {str(e)}")

    vector_store = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "ip"} 
    )
    print(f"コレクション '{collection_name}' を作成しました")

    collections = persistent_client.list_collections()
    print("現在のコレクション一覧:")
    for collection in collections:
        print(f"- {collection.name}")

    # ドキュメントをバッチで追加
    print(f"コレクション '{collection_name}' にデータをロードします")
    total_added = 0
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        vector_store.add_documents(documents=batch, embedding=embeddings)
        total_added += len(batch)
        print(f"バッチ {i//batch_size + 1} を追加しました（{len(batch)}件）")
    
    print(f"\n合計 {total_added} 件のドキュメントを追加しました")

except Exception as e:
    print(f"Chromaデータベースの操作中にエラーが発生しました: {str(e)}")


既存のコレクション 'mycollection_200' を削除しました
コレクション 'mycollection_200' を作成しました
現在のコレクション一覧:
- mycollection_200
コレクション 'mycollection_200' にデータをロードします
バッチ 1 を追加しました（500件）
バッチ 2 を追加しました（500件）
バッチ 3 を追加しました（500件）
バッチ 4 を追加しました（500件）
バッチ 5 を追加しました（500件）
バッチ 6 を追加しました（500件）
バッチ 7 を追加しました（375件）

合計 3375 件のドキュメントを追加しました


### Retrieverの準備

In [26]:
# Retrieverの準備
if search_type == "similarity":
    retriever = vector_store.as_retriever(
        search_type=search_type,
        search_kwargs={"k": top_k}
    )
elif search_type == "similarity_score_threshold":
    retriever = vector_store.as_retriever(
        search_type=search_type,
        search_kwargs={"score_threshold": score_threshold, "k": top_k}
    )
else:
    raise ValueError(f"サポートされていないsearch_type: {search_type}。'similarity'または'similarity_score_threshold'を使用してください。")

### PromptTemplateの準備

In [27]:
from langchain_core.prompts import ChatPromptTemplate

# PromptTemplateの準備
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや途中経過は出力しないでください。context の中の source のファイル名には、content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: {context}"),
        ("human", "question: {input}"),
    ]
)

### RAGチェーンの準備

In [28]:
from langchain_core.runnables import RunnablePassthrough

# RAGチェーンの準備
rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | (lambda x: print(f"\n=== Generated Prompt ===\n{x}") or x)  # プロンプトを表示
    | llm
)

### 質問応答

In [29]:
# 質問応答
response = rag_chain.invoke(query)
print(response.content)
print(response)


=== Generated Prompt ===
messages=[SystemMessage(content='あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや途中経過は出力しないでください。context の中の source のファイル名には、content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: [Document(metadata={\'source\': \'data\\\\Wikipedia-高町なのは.pdf\'}, page_content=\'。ヴィータとの戦闘でレイジングハートが⼤きく破損し窮地に陥るが、助けに来たフェイトやユーノ、アースラの⾯々の⼿助けを借りて戦闘の場から離脱する。だが離脱直前、ヴォルケンリッターのシャマルに不意をつかれてリンカーコア（魔導師の魔⼒の源）を奪われる。しかし命に別状はなく、魔⼒もすぐに回復した\'), Document(metadata={\'source\': \'data\\\\Wikipedia-高町なのは.pdf\'}, page_content=\'。⼩説版ではレイジングハートが⾃動発動しており、これでフェイトのサンダーレイジを防いでいる（ただし、完全には防げず、なのはのバリアジャケットに少し焦げ痕がついた）。なお、なのははこの魔法で防いだ直後に、ディバインバスター･フルバーストでフェイトに反撃している。ラウンドシールド(Round Shield)魔法陣を使⽤した円形の盾を作り出す防御魔法で、シールド系防御魔法に分類される\'), Document(metadata={\'source\': \'data\\\\Wikipedia-高町なのは.pdf\'}, page_content=\'。『StrikerS』ではさらに威⼒が強化され、レイジン

## ソースつきのRAGの作成

### ソース付きRAGチェーンの準備

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# コンテキストのフォーマット
def format_docs(docs):
    #return "\n\n".join(doc.page_content for doc in docs)
    #return "\n\n".join(f"[source]: {doc.metadata['source']}\n[content]: {doc.page_content}" for doc in docs)
    #return "\n\n".join(f"[metadata]: {doc.metadata}\n[content]: {doc.page_content}" for doc in docs)
    return "\n\n".join(f"{doc}" for doc in docs)


# Generationチェーンの準備
gemeration_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_template
    #| (lambda x: print(f"\n=== Generated Prompt ===\n{x}") or x)  # プロンプトを表示
    | llm
    | StrOutputParser()
)

# Retrieverチェーンの準備
retrieve_chain = (lambda x: x["input"]) | retriever

# ソース付きRAGチェーンの準備
rag_chain_with_source = RunnablePassthrough.assign(context=retrieve_chain).assign(
    answer=gemeration_chain
)

### 質問応答

In [31]:
query="高町なのはがユーノから譲り受けたものは何ですか？"

In [32]:
# 質問応答の結果を見やすくフォーマットする関数
def format_rag_response(response):
    print("質問:", response["input"])
    print("\n回答:", response["answer"])
    print("\n参照コンテキスト:")
    for i, doc in enumerate(response["context"], 1):
        print(f"\n--- ドキュメント {i} ---")
        print(f"ソース: {doc.metadata['source']}")
        print(f"文字数: {len(doc.page_content)}")
        print(f"内容: {doc.page_content[:1000]}...")  # 長すぎる場合は省略

# 質問応答
response = rag_chain_with_source.invoke({"input": query})
format_rag_response(response)
print(response)


質問: 高町なのはがユーノから譲り受けたものは何ですか？

回答: 高町なのはがユーノから譲り受けたものは、インテリジェントデバイス「レイジングハート」です。

参照コンテキスト:

--- ドキュメント 1 ---
ソース: data\Wikipedia-高町なのは.pdf
文字数: 152
内容: 。敵対していたフェイトやヴィータなどの⼈物とも、幾度となく互いの信念をかけた戦いを繰り返した結果、最終的には分かり合うことが出来ており、彼⼥達と良き友となっている。使⽤デバイス（魔法の杖）はインテリジェントデバイス「レイジングハート」。ユーノから譲り受けたものだが、彼がこのデバイスを⼿に⼊れた経緯は不明...

--- ドキュメント 2 ---
ソース: data\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf
文字数: 167
内容: 。新たなリーダーとなったひめなは静⾹に⼿を組むことを持ちかけるが拒否される。ひめなの指⽰のもと、⽻根の神楽燦と遊狩ミユリがちはるの腕を切り落とし、⽯を強奪する。那由他は伯⽗である灯花の⽗に⼿はずを整えてもらい、同じく⽗の⾏⽅を探っている離婚した⺟と再会。⽗に関する調査資料を⼊⼿し、「湯国市」に⼿がかりがあるとわかって⾏動しようとする...

--- ドキュメント 3 ---
ソース: data\Wikipedia-魔法少女リリカルなのは.pdf
文字数: 182
内容: 。彼がこの世界に来た理由。それは彼が発掘したロストロギア（異世界に存在した⾼度な魔法技術の遺産）「ジュエルシード」が散らばってしまったためであった。成り⾏きから事情を知ったなのはは、ユーノと共にジュエルシードを集め、封印する⼿伝いをすることになる。そして次第になのははユーノだけではなく、⾃分のために魔法の世界に関わっていく決意を固め、その秘めた⼒を開花させていく...

--- ドキュメント 4 ---
ソース: data\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf
文字数: 182
内容: 。神浜市内の⼀般⼈は魔法の⼒によってネオマギウスに従わない限り飲⾷ができない状況となり、⽣殺与奪の権利を握られることになる。那由他はフォークロアの4⼈、そして太助と湯国市に赴き、以前訪れた時⼥⼀族同様魔法少⼥への迫害に遭う。⽗か

## RAGASによる評価


### 評価用 LLM と埋め込みモデルの準備

In [34]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_core.outputs import LLMResult
def my_finished_parser(response: LLMResult) -> bool:
    #print(response)
    return True
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("OPENAI_API_KEY")), is_finished_parser=my_finished_parser)
#evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("OPENAI_API_KEY")))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.getenv("OPENAI_API_KEY")))

### 質問・正解・コンテキスト・回答の準備

In [35]:
# 質問
question = [
    "高町なのはがユーノから譲り受けたものは何ですか？",
    "高町なのはには兄弟姉妹はいますか？名前はわかりますか？",
    "高町なのはは、フェイトのことをどのように呼んでいますか？",
    "四葉家の次期当主候補は誰ですか？",
    "時空管理局のアースラの艦長は誰ですか？",
    "ユーノ・スクライアは、普段どのような動物の姿で過ごしていますか？",
    "フェイト・テスタロッサのデバイス（魔法の杖）の名前は？",
    "高町なのはが小学生のときに通っていた学校は？",
    "高町なのはとユーノが回収しようとしているものは何？？",
    "古代ベルカの諸王時代に存在した王家とは？",
    "魔法少女リリカルなのはの主人公は？",
    "魔法少女まどか☆マギカの主人公の名前は？",
    "鹿目まどかの声は誰が担当していますか？",
    "鹿目まどかが魔法少女となったときに使う武器は？",
    "リリカルなのはで、自立判断し英語でしゃべるパートナーとして描かれているものは何ですか？",
    "聖王のゆりかごとは何ですか？",
    "スターライトブレイカーは誰の魔法ですか？",
    "ニュータイプアニメアワード2011では女性キャラクター賞で1位を獲得したキャラクターは？",
    "レリックの発掘と同時に現れた謎の機械兵器群は何ですか？"
]

# 正解
ground_truth = [
    "高町なのはがユーノから譲り受けたものはインテリジェントデバイスのレイジングハートです",
    "高町なのはには、兄の恭也と姉の美由希がいます。",
    "高町なのはは、フェイトのことを公の場では「フェイト隊長」と呼ぶこともあるが、プライベートでは「フェイトちゃん」と呼んでいます。",
    "四葉家の次期当主候補は司波深雪です。",
    "時空管理局のアースラの艦長は、リンディ・ハラオウンです。",
    "ユーノ・スクライアは、普段フェレットのような動物の姿で過ごしています。",
    "フェイト・テスタロッサののデバイス（魔法の杖）の名前はバルディッシュです。",
    "高町なのはが小学生のときに通っていた学校は、私立聖祥大附属小学校です。",
    "高町なのはとユーノが回収しようとしているものはジュエルシードです。",
    "古代ベルカの諸王時代に存在した王家は、聖王家です。",
    "魔法少女リリカルなのはの主人公は、高町なのはです。",
    "魔法少女まどか☆マギカの主人公の名前は、鹿目まどかです。",
    "鹿目まどかの声は、悠木碧（ゆうき あおい）が担当しています。",
    "鹿目まどかが魔法少女となったときに使う武器は、弓です。",
    "魔法少女リリカルなのはで、自立判断し英語でしゃべるパートナーとして描かれているものは、「魔法の杖」です。",
    "聖王のゆりかごとは、古代ベルカの遺産の一つ。その正体は旧暦において一度は世界を滅ぼしたとされる強大な質量兵器で、全長数kmの巨大飛行戦艦です。",
    "スターライトブレイカーは、なのは最大最強の切り札であり、彼女の代名詞ともなっている魔法。なのは本人（+カートリッジ）の魔力に加えて、なのは個人の希少スキル「魔力収束」をもって戦場に漂う残留魔力をも呼び集めて特大の一撃を放つ集束砲撃魔法です。",
    "ニュータイプアニメアワード2011では女性キャラクター賞で1位を獲得したキャラクターは、暁美ほむらです。",
    "レリックの発掘と同時に現れた謎の機械兵器群は、ガジェット・ドローンです。"
]

# コンテキストと回答の生成
contexts = []
answer = []
for q in question:
    print("question:", q)
    response = rag_chain_with_source.invoke({"input": q})
    contexts.append([x.page_content for x in response["context"]])
    print(f"answer: {response['answer']}")
    answer.append(response["answer"])

#print("contexts:", contexts)
#print("answer:", answer)

question: 高町なのはがユーノから譲り受けたものは何ですか？
answer: 高町なのはがユーノから譲り受けたものは、インテリジェントデバイス「レイジングハート」です。
question: 高町なのはには兄弟姉妹はいますか？名前はわかりますか？


Task exception was never retrieved
future: <Task finished name='Task-10' coro=<AsyncClient.aclose() done, defined at g:\lab\ragas-langchain-chroma\.venv\Lib\site-packages\httpx\_client.py:1978> exception=AssertionError()>
Traceback (most recent call last):
  File "g:\lab\ragas-langchain-chroma\.venv\Lib\site-packages\httpx\_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "g:\lab\ragas-langchain-chroma\.venv\Lib\site-packages\httpx\_transports\default.py", line 406, in aclose
    await self._pool.aclose()
  File "g:\lab\ragas-langchain-chroma\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "g:\lab\ragas-langchain-chroma\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 343, in _close_connections
    with AsyncShieldCancellation():
  File "g:\lab\ragas-langchain-chroma\.venv\Lib\site-packages\httpcore\_synchronization.py", line 214, in __enter__
    se

answer: context にある情報だけで回答が可能です。高町なのはには兄と姉がいます。兄の名前は恭也、姉の名前は美由希です。
question: 高町なのはは、フェイトのことをどのように呼んでいますか？
answer: context によると、高町なのははフェイトのことをプライベートでは「フェイトちゃん」と呼んでいます。
question: 四葉家の次期当主候補は誰ですか？
answer: 四葉家の次期当主候補は深雪です。
question: 時空管理局のアースラの艦長は誰ですか？
answer: 情報がないので回答できません。
question: ユーノ・スクライアは、普段どのような動物の姿で過ごしていますか？
answer: 情報がないので回答できません。
question: フェイト・テスタロッサのデバイス（魔法の杖）の名前は？
answer: フェイト・テスタロッサのデバイス（魔法の杖）の名前は「バルディッシュ」です。
question: 高町なのはが小学生のときに通っていた学校は？
answer: 情報がないので回答できません。
question: 高町なのはとユーノが回収しようとしているものは何？？
answer: 高町なのはとユーノが回収しようとしているものは「ジュエルシード」です。
question: 古代ベルカの諸王時代に存在した王家とは？
answer: 情報がないので回答できません。
question: 魔法少女リリカルなのはの主人公は？
answer: 魔法少女リリカルなのはの主人公は、高町なのはです。
question: 魔法少女まどか☆マギカの主人公の名前は？
answer: 魔法少女まどか☆マギカの主人公の名前は、鹿目まどかです。
question: 鹿目まどかの声は誰が担当していますか？


  5%|▌         | 1/19 [45:42<13:42:53, 2742.95s/it]


InternalServerError: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text-black-dar

### データセットの準備

In [194]:
from datasets import Dataset 

# データセットの準備
ds = Dataset.from_dict(
    {
        "question": question,
        "answer": answer,
        "contexts": contexts,
        "ground_truth": ground_truth,
    }
)

### 評価実行
「faithfulness」「answer_relevancy」「context_precision」「context_recall」の4つのメトリックス


In [ ]:
from ragas import evaluate
from ragas.metrics import AnswerCorrectness, AnswerSimilarity, Faithfulness, AnswerRelevancy, ContextPrecision, ContextRecall

metrics = [
    AnswerCorrectness(llm=evaluator_llm,embeddings=evaluator_embeddings),
    #AnswerSimilarity(embeddings=evaluator_embeddings),
    #Faithfulness(llm=evaluator_llm),
    #AnswerRelevancy(llm=evaluator_llm,embeddings=evaluator_embeddings),
    #ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm)
]
#metrics = [
#    ContextRecall(llm=evaluator_llm)
#]
# 評価の実行
result = evaluate(
    dataset=ds,
    metrics=metrics
)
print(result)

In [ ]:
import matplotlib.pyplot as plt
import math
plt.rcParams['font.family'] = 'MS Gothic'

# 結果データの準備
answer_correctness_avr = math.floor(sum(result['answer_correctness'])/len(result['answer_correctness']) * 1000) / 1000
context_recall_avr = math.floor(sum(result['context_recall'])/len(result['context_recall']) * 1000) / 1000


metrics = {
    'Answer Correctness': answer_correctness_avr,
    'Context Recall': context_recall_avr
}

# プロットの作成
plt.figure(figsize=(10, 6))
bars = plt.bar(list(metrics.keys()), list(metrics.values()))

# グラフの装飾
plt.title('RAGASによる評価結果', pad=20)
plt.ylim(0, 1.0)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 数値を棒グラフの上に表示
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}',
             ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import math
plt.rcParams['font.family'] = 'MS Gothic'

# 結果データの準備
answer_correctness_avr = math.floor(sum(result['answer_correctness'])/len(result['answer_correctness']) * 1000) / 1000
answer_similarity_avr = math.floor(sum(result['semantic_similarity'])/len(result['semantic_similarity']) * 1000) / 1000
faithfulness_avr = math.floor(sum(result['faithfulness'])/len(result['faithfulness']) * 1000) / 1000
answer_relevancy_avr = math.floor(sum(result['answer_relevancy'])/len(result['answer_relevancy']) * 1000) / 1000
context_precision_avr = math.floor(sum(result['context_precision'])/len(result['context_precision']) * 1000) / 1000
context_recall_avr = math.floor(sum(result['context_recall'])/len(result['context_recall']) * 1000) / 1000


metrics = {
    'Answer Correctness': answer_correctness_avr,
    'Answer Similarity': answer_similarity_avr,
    'Faithfulness': faithfulness_avr,
    'Answer Relevancy': answer_relevancy_avr,
    'Context Precision': context_precision_avr,
    'Context Recall': context_recall_avr
}

# プロットの作成
plt.figure(figsize=(10, 6))
bars = plt.bar(list(metrics.keys()), list(metrics.values()))

# グラフの装飾
plt.title('RAGASによる評価結果', pad=20)
plt.ylim(0, 1.0)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 数値を棒グラフの上に表示
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}',
             ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
metrics = {
    'Answer Correctness': answer_correctness_avr,
    'Context Recall': context_recall_avr
}

# プロットの作成
plt.figure(figsize=(10, 6))
bars = plt.bar(list(metrics.keys()), list(metrics.values()))

# グラフの装飾
plt.title('RAGASによる評価結果', pad=20)
plt.ylim(0, 1.0)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 数値を棒グラフの上に表示
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}',
             ha='center', va='bottom')

plt.tight_layout()
plt.show()